# Problem - 2 : Gbike Bicycle Rental

-----------------------------------------------------------------------------------------------

### Importing Libraries

In [ ]:
import sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import poisson

-----------------------------------------------------------------------------------------------

### Setting the Problem Parameters

In [ ]:
class gbr:
    @staticmethod
    def max_bikes():
        return 20
    
    @staticmethod
    def γ():
        return 0.9
    
    @staticmethod
    def credit_reward():
        return 10
    
    @staticmethod
    def moving_reward():
        return -2

-----------------------------------------------------------------------------------------------

### Defining the Poisson Class

In [ ]:
class poisson_:
    def __init__(self, λ):
        self.λ = λ
        ε = 0.01
        
        # [α , β] is the range of n's for which the pmf value is above ε
        self.α = 0
        state = 1
        self.vals = {}
        summ = 0
        
        while(1):
            if state == 1:
                temp = poisson.pmf(self.α, self.λ) 
                if(temp <= ε):
                    self.α+=1
                else:
                    self.vals[self.α] = temp
                    summ += temp
                    self.β = self.α+1
                    state = 2
            elif state == 2:
                temp = poisson.pmf(self.β, self.λ)
                if(temp > ε):
                    self.vals[self.β] = temp
                    summ += temp
                    self.β+=1
                else:
                    break    
        
        # normalizing the pmf, values of n outside of [α, β] have pmf = 0
        added_val = (1-summ)/(self.β-self.α)
        for key in self.vals:
            self.vals[key] += added_val
            
    def f(self, n):
        try:
            Ret_value = self.vals[n]
        except(KeyError):
            Ret_value = 0
        finally:
            return Ret_value

### Defining the Class for holding the Location

In [ ]:
class location:
    def __init__(self, req, ret):
        self.α = req                             #value of lambda for requests
        self.β = ret                             #value of lambda for returns
        self.poissonα = poisson_(self.α)
        self.poissonβ = poisson_(self.β)

-----------------------------------------------------------------------------------------------

### Function Definitions

In [ ]:
def apply_action(state, action):
    return [max(min(state[0] - action, gbr.max_bikes()),0) , max(min(state[1] + action, gbr.max_bikes()),0)]

#### Expected Reward

In [ ]:
"""
NOTE -
    state  : It's a pair of integers, # of bikes at A and at B
    action : # of bikes transferred from A to B,  -5 <= action <= 5 
"""

def expected_reward(state, action):
    global value
    
    # reward
    ψ = 0
    
    new_state = apply_action(state, action)
    
    # adding reward for moving bikes from one location to another (which is negative) 
    ψ = ψ + gbr.moving_reward() * abs(action)
    
    #there are four discrete random variables which determine the probability distribution of the reward and next state
    """
        Aα : sample of bikes requested at location A
        Aβ : sample of bikes returned at location A
        Bα : sample of bikes requested at location B
        Bβ : sample of bikes returned at location B
        ζ  : probability of this event happening
    """
    for Aα in range(A.poissonα.α, A.poissonα.β):
        for Bα in range(B.poissonα.α, B.poissonα.β):
            for Aβ in range(A.poissonβ.α, A.poissonβ.β):
                for Bβ in range(B.poissonβ.α, B.poissonβ.β):
                    
                    # all four variables are independent of each other
                    ζ = A.poissonα.vals[Aα] * B.poissonα.vals[Bα] * A.poissonβ.vals[Aβ] * B.poissonβ.vals[Bβ]
                    
                    valid_requests_A = min(new_state[0], Aα)
                    valid_requests_B = min(new_state[1], Bα)
                    
                    rew = (valid_requests_A + valid_requests_B)*(gbr.credit_reward())
                    
                    #calculating the new state based on the values of the four random variables
                    new_s = [0,0]
                    new_s[0] = max(min(new_state[0] - valid_requests_A + Aβ, gbr.max_bikes()),0)
                    new_s[1] = max(min(new_state[1] - valid_requests_B + Bβ, gbr.max_bikes()),0)
                    
                    #Bellman's equation
                    ψ += ζ * (rew + gbr.γ() * value[new_s[0]][new_s[1]])
    return ψ

#### Policy Evaluation

In [ ]:
def policy_evaluation():
    global value
    
    # here policy_evaluation has a static variable ε whose values decreases over time
    ε = policy_evaluation.ε
    
    policy_evaluation.ε /= 10 
    
    while(1):
        δ = 0
        for i in range(value.shape[0]):
            for j in range(value.shape[1]):  # value[i][j] denotes the value of the state [i,j]
                old_val = value[i][j]
                value[i][j] = expected_reward([i,j], policy[i][j])
                
                δ = max(δ, abs(value[i][j] - old_val))
                print('-', end = '')
                sys.stdout.flush()
                    
        print(δ)
        sys.stdout.flush()
    
        #break            # Uncomment for Value Iteration [1 Sweep only]

        if δ < ε:
            break

#### Policy Improvement

In [ ]:
def policy_improvement():
    global policy
    
    policy_stable = True
    for i in range(value.shape[0]):
        for j in range(value.shape[1]):
            old_action = policy[i][j]
            
            max_act_val = None
            max_act = None
            
            τ12 = min(i,5)       # if I have say 3 bikes at the first location, then I can atmost move 3 from 1 to 2
            τ21 = -min(j,5)      # if I have say 2 bikes at the second location, then I can atmost move 2 from 2 to 1
            
            for act in range(τ21,τ12+1):
                σ = expected_reward([i,j], act)
                if max_act_val == None:
                    max_act_val = σ
                    max_act = act
                elif max_act_val < σ:
                    max_act_val = σ
                    max_act = act
                
            policy[i][j] = max_act
            
            if old_action!= policy[i][j]:
                policy_stable = False
    
    return policy_stable

-----------------------------------------------------------------------------------------------

### Saving the Policy and Value Results as Heatmaps

In [ ]:
def save_policy():
    save_policy.counter += 1
    ax = sns.heatmap(policy, linewidth=0.5, cmap="Greens")
    ax.invert_yaxis()
    plt.savefig('policy'+str(save_policy.counter)+'.svg')
    plt.close()

In [ ]:
def save_value():
    save_value.counter += 1
    ax = sns.heatmap(value, linewidth=0.5, cmap="Greens")
    ax.invert_yaxis()
    plt.savefig('value'+ str(save_value.counter)+'.svg')
    plt.close()

-----------------------------------------------------------------------------------------------

### Initialising the Variables for Functions and Setting the Parameters

#### For Location

In [ ]:
A = location(3,3)
B = location(4,2)

#### For Matrices of Values and Policies

In [ ]:
# NOTE : Initial policy has zero value for all states

value = np.zeros((gbr.max_bikes()+1, gbr.max_bikes()+1))
policy = value.copy().astype(int)

#### For ε

In [ ]:
policy_evaluation.ε = 50

#### For the Counter of both Policy and Value

In [ ]:
save_policy.counter = 0
save_value.counter = 0

-----------------------------------------------------------------------------------------------

### Running the Functions in Order for Obtaining the Final Output

In [ ]:
while(1):
    policy_evaluation()
    ρ = policy_improvement()
    save_value()
    save_policy()
    if ρ == True:
        break

print("\n THE END \n")

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------190.7422667431856
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------132.3288138607936
----------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------4.429149938857279
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------5.27798779239663
-----------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------0.2010214703025781
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------0.1662931285351874
--------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------0.007000437207864252
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------0.005740224176975062
----------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------